In [15]:
from unsloth import FastLanguageModel
import os
import json
from datasets import Dataset
from transformers import TextStreamer
import torch
FILE_PATH=os.getcwd()
CODE_PATH= os.path.join(FILE_PATH, '..')
DATA_PATH=os.path.join(CODE_PATH, 'data')
MODELS_PATH=os.path.join(CODE_PATH, 'models')
FINETUNED_PATH=os.path.join(MODELS_PATH, 'finetuned')

PROCESSED_DATA_PATH=os.path.join(DATA_PATH, 'processed')
RAW_DATA_PATH=os.path.join(DATA_PATH, 'raw')
USED_DATA_PATH=os.path.join(DATA_PATH, 'used')

tokenizer=None
def pre_apply_chat_template(example):  
    conversations = example["text"]  
    text = tokenizer.apply_chat_template(conversations, tokenize=False, add_generation_prompt=False)  
    return {"text": text}  
def pre_apply_chat_template_gen(example):  
    conversations = example["text"][:-1]  
    text = tokenizer.apply_chat_template(conversations, tokenize=False, add_generation_prompt=True)  
    return {"text": text}  
def pre_apply_chat_template_gen_tokenize(example):  
    conversations = example["text"][:-1]  
    text = tokenizer.apply_chat_template(conversations, tokenize=True, add_generation_prompt=True)  
    return {"text": text}    

### Out of Memory if GPU, Internal error if CPU

In [14]:
with open(os.path.join(USED_DATA_PATH, 'used_dataset_sys_use_ass.json'), 'r') as f:
    used_dataset = json.load(f)
    
dataset = Dataset.from_list(used_dataset)  

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/home/kids/Linux_Coding/Smart-File-Finder/models/finetuned/Mistral_d66ea65/Mistral_d66ea65_adapters",
    max_seq_length=2048,
    load_in_4bit=True,
    # llm_int8_enable_fp32_cpu_offload = True,
    device_map="cpu",  # Force CPU usage
)


==((====))==  Unsloth 2025.5.6: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


RuntimeError: !handles_.at(i) INTERNAL ASSERT FAILED at "/pytorch/c10/cuda/CUDACachingAllocator.cpp":396, please report a bug to PyTorch. 

In [7]:
FastLanguageModel.for_inference(model) 

# Apply the chat template to the dataset
dataset_gen = dataset.map(pre_apply_chat_template_gen)

Map:   0%|          | 0/850 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'apply_chat_template'

In [9]:
type(tokenizer)

NoneType

In [6]:
inputs = tokenizer(
    pre_apply_chat_template_gen(dataset_gen[0],tokenizer)['text'],
    return_tensors="pt",
    # add_special_tokens=False, 
    padding=True,
).to("cuda")
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(inputs['input_ids'], attention_mask = inputs['attention_mask'], streamer=text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)
_ = model.generate(torch.tensor([pre_apply_chat_template_gen_tokenize(dataset_gen[0],tokenizer)['text']]).to("cuda"), 
                    streamer=text_streamer,
                    max_new_tokens = 128, 
                    pad_token_id = tokenizer.eos_token_id)

print(_)
pass


NameError: name 'dataset_gen' is not defined